## Needed package

In [1]:
from pulp import *
import pandas as pd
import numpy as np
from datetime import datetime
import concurrent.futures
from tqdm import tqdm
from loguru import logger

## Functions

* CustomizedLog: record log
* GetConstraintInfo:
    * make relationship of demand and bom dictionary
    * use the relationship to generate constraints
* generate_LP_results: have result of preemptive goal programming into dataframe

In [2]:
def CustomizedLog(file_log='./log/file.log'):  # add: log folder
    logger.add(file_log,
               format="{time:YYYY-MM-DD HH:mm:ss|\
                       [{level}]|{file}|\
                       {function}()-[{line}]|\
                       {message}}",
                retention="10 days",
                encoding="utf-8",
                level="INFO")
    return logger

In [3]:
#### functions
class GetConstraintInfo:
    def __init__(self, dem_df, bom_df, con_df, top_col, down_col):
        self.dem_df = dem_df
        self.bom_df = bom_df
        self.con_df = con_df
        self.top_col = top_col
        self.down_col = down_col
    
    def output_bom(self, key, group):
        bom_sub = dict()
        group = pd.merge(group, self.dem_df, on=self.top_col, how='inner')
        bom_sub[key] = group.set_index(self.top_col)['EXTENDED_QUANTITY'].to_dict()
        return bom_sub

    def get_bom_dict(self):
        #### known variables
        bom = {}
        with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(self.output_bom, key, group) for key, group in self.bom_df.groupby(self.down_col)]
            for fut in tqdm(concurrent.futures.as_completed(futures)):
                bom.update(fut.result()) # result() 若 function 僅回傳一種值則不需要索引
        return bom

    def get_constraints_dict(self):
        model = LpProblem("Max-Shipment-Problem", LpMaximize)
        bom_dict = self.get_bom_dict()
        for down in self.con_df[self.down_col].values:
            main_func = [- bom_dict[down][up]*self.dem_df[self.dem_df[self.top_col]==up]['DEMAND_QTY'].values[0]*z[up] for up, _ in bom_dict[down].items()]
            main_func += [self.con_df[self.con_df[self.down_col]==down]['ASSIGNABLE_AMOUNT'].values[0]]
            model += lpSum(main_func) >= 0
        constraints_dict = model.constraints
        return constraints_dict

def generate_LP_results(z):
    output = []
    for i in z:
        var_output = {
            'TOP_SKU':i,
            'ontime_shipment':z[i].varValue
        }
        output.append(var_output)
    output_df = pd.DataFrame.from_records(output)
    return output_df

## Data Definition:
For example:
* dem_df: 
    * 90-A is composed of one Comp1 and one Comp2
    * the amount of order of 90-A is 1000 
* bom_df:
    * 90-A is composed of one Comp1 and one Comp2
* con_df:
    * the assignable amount of Comp1 is 2200
    * after calculating, the amount of Comp1 in demand is 3700

In [4]:
logger = CustomizedLog()

CRED = '\033[91m'
CEND = '\033[0m'

file_name = 'pulp_pgp_example'
all_df = pd.read_excel(f'./Data/{file_name}.xlsx', sheet_name=None)
dem_df = all_df['demand']
bom_df = all_df['bom']
con_df = all_df['constraint']

print(CRED+'dem_df:'+CEND, dem_df, sep='\n')
print(CRED+'bom_df:'+CEND, bom_df, sep='\n')
print(CRED+'con_df:'+CEND, con_df, sep='\n')

dem_df:
  TOP_SKU  DEMAND_QTY  PRIORITY  Comp1  Comp2  Comp3
0    90-A        1000         1      1      1      0
1    90-B         700         1      1      1      1
2    90-C         500         2      0      1      0
3    90-D         300         2      1      0      0
4    90-E         900         3      1      0      0
5    90-F         800         3      1      0      1
bom_df:
  TOP_SKU COMPONENT_ITEM  EXTENDED_QUANTITY
0    90-A          Comp1                  1
1    90-B          Comp1                  1
2    90-D          Comp1                  1
3    90-E          Comp1                  1
4    90-F          Comp1                  1
5    90-A          Comp2                  1
6    90-B          Comp2                  1
7    90-C          Comp2                  1
8    90-B          Comp3                  1
9    90-F          Comp3                  1
con_df:
  COMPONENT_ITEM  DEMAND_AMOUNT  ASSIGNABLE_AMOUNT
0          Comp1           3700               2200
1          Comp2   

## Preemptive goal programming:
1. Considering the priority of demand, which means priority=1 should be shipped first if possible.
2. Set $Z_{TOP\:SKU}$ is binary unknwon variable. 
      
    $$
    Z_{TOP\:SKU}
    \left\{
    \begin{align} 
    &=1,& able\:to\:be\:shipped  \\
    &=0,& unable\:to\:be\:shipped  
    \end{align}
    \right.
    $$
      
3. Objective function: the larger timely shipment, the better  
      
    $$\max\sum_{all\:TOP\:SKU}DEMAND_{TOP\:SKU}*Z_{TOP\:SKU}$$
      
4. Constraints: the assignable amount is the ceiling of timely shipment  
      
    $$
    AMOUNT_{COMPONENT\:ITEM}
    -\sum_{all\:TOP\:SKU\:using\:COMPONENT\:ITEM}
    BOM_{\left(COMPONENT\:ITEM, TOP\:SKU\right)}DEMAND_{TOP\:SKU}*Z_{TOP\:SKU}\ge0
    $$
      
5. Reference:
    * [Preemptive Goal Programming](https://www.riverware.org/PDF/RiverWare/documentation/80_prerelease/index.html#page/Optimization/OptimMath.3.1.html)
    * [pulp example of considering priority](https://www.supplychaindataanalytics.com/multi-objective-linear-optimization-with-pulp-in-python/)

In [5]:
#### create all unknown variables
f_sku = dem_df.TOP_SKU.unique()
z = LpVariable.dicts("z",
                    (sku for sku in f_sku),
                     lowBound = 0,
                     upBound = 1,
                     cat = 'Integer')

#### get all needed material before solving model
priority_lst = list(np.sort(dem_df.PRIORITY.unique()))
constraints_dict = GetConstraintInfo(dem_df, bom_df, con_df, 'TOP_SKU', 'COMPONENT_ITEM').get_constraints_dict()

#### create objective function
for i in priority_lst:
    logger.info(f'priority={i}')
    model = LpProblem("Max-Shipment-Problem", LpMaximize)
    for k, v in constraints_dict.items(): model += v
    f_sku = dem_df[dem_df.PRIORITY==i]['TOP_SKU']
    lst = [z[up]*dem_df[dem_df['TOP_SKU']==up]['DEMAND_QTY'].values[0] for up in f_sku]
    model += lpSum(lst)
    model.solve()
    assert LpStatus[model.status] == 'Optimal'
    if priority_lst.index(i)<(len(priority_lst)-1):
        model += lpSum([model.objective]) >= model.objective.value()
        constraints_dict = model.constraints

3it [00:00, 163.64it/s]
2021-10-07 16:02:41.626 | INFO     | __main__:<module>:15 - priority=1
2021-10-07 16:02:41.648 | INFO     | __main__:<module>:15 - priority=2
2021-10-07 16:02:41.670 | INFO     | __main__:<module>:15 - priority=3


## Result
Focus on column: ontime_shipment  
The dataframe means:
1. 90-A, 90-C, 90-D, 90-E can be shipped on time.
2. 90-B, 90-F can't be shipped on time because of assignable limitation amount.

In [6]:
output_df = generate_LP_results(z)
dem_df = pd.merge(dem_df, output_df, on='TOP_SKU', how='left')
dem_df.ontime_shipment = dem_df.ontime_shipment.fillna(0)
print(CRED+'order ontime result:'+CEND, dem_df, sep='\n')

order ontime result:
  TOP_SKU  DEMAND_QTY  PRIORITY  Comp1  Comp2  Comp3  ontime_shipment
0    90-A        1000         1      1      1      0              1.0
1    90-B         700         1      1      1      1              0.0
2    90-C         500         2      0      1      0              1.0
3    90-D         300         2      1      0      0              1.0
4    90-E         900         3      1      0      0              1.0
5    90-F         800         3      1      0      1              0.0
